# 용어 정리

## Reranker
- Reranker는 검색 쿼리와 문서 간의 관련성 점수를 계산하는 언어 모델 유형
- 크고 복잡한 문서의 쿼리를 처리해야 하는 경우, 임베딩 기반 검색을 진행한 후 Reranker를 사용하는 것이 성능 개선에 도움
- Retriever -> Reranker (two-stage)

## Reranker for RAG (Advanced RAG)
- Advanced RAG는 쿼리와 관련한 문서를 유사도 검색을 기반으로 검색하는 데서 더 나아가 관련 문서를 정확하고 효율적으로 검색하여 답변 성능을 높인 RAG 방식
- 메타 데이터 추가, 데이터 구조화, 파인 튜닝, 리랭커, 프롬프트 컴프레션 등

In [2]:
!pip install -qU cohere langchain langchain-cohere langchain_community faiss-cpu

### 🗝️ Cohere API Key

https://dashboard.cohere.com/api-keys

In [3]:
import os

os.environ["COHERE_API_KEY"] = "uS..." # 발급받은 API Key 를 복사 + 붙여넣기

## Cohere Embedding model - Retriever

#### 0. Retriever에 사용할 임베딩 모델

- Cohere Embed Models : https://docs.cohere.com/docs/cohere-embed

In [5]:
from langchain_cohere import CohereEmbeddings

cohere_embeddings = CohereEmbeddings(model="embed-english-light-v3.0") # 다국어 : embed-multilingual-light-v3.0, ...
text = "This is a test document."

embedding_result = cohere_embeddings.embed_query(text)
print(embedding_result)

[-0.09338379, 0.0871582, -0.03326416, 0.01953125, 0.07702637, 0.034729004, -0.058380127, -0.031021118, -0.030517578, -0.055999756, 0.050842285, -0.006752014, 0.038391113, -0.0014362335, -0.041137695, -0.008880615, 0.026000977, -0.023010254, 0.05456543, -0.03366089, 0.055633545, 0.028579712, -0.068603516, 0.03970337, -0.06677246, 0.06732178, -0.013053894, -0.0060920715, 0.038116455, 0.057800293, 0.048736572, 0.026855469, 0.009849548, 0.08312988, 0.073791504, 0.01663208, -0.0871582, 0.01802063, -0.0020828247, -0.0031356812, 0.039978027, -0.03164673, 0.009796143, 0.011375427, 0.0068855286, 0.092285156, 0.05218506, -0.060943604, 0.038269043, -0.018218994, -0.04510498, -0.0847168, 0.008300781, -0.060058594, 0.0012111664, 0.05102539, 0.05218506, -0.047210693, -0.051239014, -0.044158936, -0.058166504, 0.07849121, -0.019165039, 0.06451416, 0.024887085, 0.011405945, -0.03768921, -0.018814087, -0.06829834, -0.052825928, -0.019104004, -0.021194458, 0.043518066, 0.07525635, 0.082336426, 0.00376510

In [6]:
print("Embedding Dim :",len(embedding_result))

Embedding Dim : 384


#### 1. 샘플 텍스트 Load 하기

- Sample txt(Kor) : https://n.news.naver.com/mnews/article/003/0013004563

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = TextLoader("/content/samlple.txt").load()

In [8]:
documents

[Document(metadata={'source': '/content/samlple.txt'}, page_content='경기도 전역 한파특보…재난안전 1단계 대응\n입력\n2025.01.08. 오후 1:38\n수정\n2025.01.08. 오후 1:38\n8일 오후 9시 31개 시군에 한파특보 발효 예정\n[수원=뉴시스] 이병희 기자 = 경기도가 도내 31개 시군 전역에 예보된 한파특보에 따라 8일 오후 1시부터 재난안전대책본부 1단계를 가동, 선제 대응에 나섰다.\n\n기상청은 이날 오후 9시를 기해 경기도 전역에 한파특보를 발효할 예정이다. 한파주의보가 내려졌던 연천·포천·가평·파주 등 4개 시·군은 한파경보로 변경되고, 동두천·양주·의정부·남양주·여주·양평 등 6개 시·군은 한파경보가 발효된다.\n\n또 광명, 과천, 안산, 시흥, 부천, 김포, 고양, 수원, 성남, 안양, 구리, 오산, 평택, 군포, 의왕, 하남, 용인, 이천, 안성, 화성, 광주 등 경기남부지역을 비롯한 21개 지역에는 한파주의보를 발표됐다.\n\n북서쪽에서 남하하는 찬 공기의 영향으로 9~10일 아침 기온은 이날보다 5~8도가량 떨어져 대부분 지역에서 영하 10도 이하, 일부 경기북부·동부는 영하 15도 안팎이 돼 매우 추울 전망이다.\n\n도는 재난안전대책본부 1단계를 가동해 복지·상하수 분야 등 6개 반 13개 부서 13명이 분야별 대응하며 한파에 대비할 방침이다.\n\n한파 피해 최소화를 위해 ▲노인, 노숙인 등 취약계층 안전확인 강화 ▲지역자율방재단, 이·통장 등 협업을 통한 한파쉼터 운영상황 등 점검 ▲농작물 냉해 및 가축 동사 방지 등 사전 대비 ▲야외활동 자제, 부모님께 안부전화 하기 등 한파 행동요령과 안전수칙 적극 홍보를 시군에 요청했다.\n\n김동연 경기도지사는 "이번 주 갑작스러운 기온 하강에 따라 피해가 우려된다. 노약자 등 취약계층에 대한 안전은 물론 농축업 등 산업분야까지 세심하게 살펴 피해를 입지 않도록 해달라"고 당부했다.\n\n한편 한파특보가 내려질 

#### 2. 텍스트 Split 하기

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

In [10]:
texts

[Document(metadata={'source': '/content/samlple.txt'}, page_content='경기도 전역 한파특보…재난안전 1단계 대응\n입력\n2025.01.08. 오후 1:38\n수정\n2025.01.08. 오후 1:38\n8일 오후 9시 31개 시군에 한파특보 발효 예정'),
 Document(metadata={'source': '/content/samlple.txt'}, page_content='[수원=뉴시스] 이병희 기자 = 경기도가 도내 31개 시군 전역에 예보된 한파특보에 따라 8일 오후 1시부터 재난안전대책본부 1단계를 가동, 선제 대응에 나섰다.'),
 Document(metadata={'source': '/content/samlple.txt'}, page_content='기상청은 이날 오후 9시를 기해 경기도 전역에 한파특보를 발효할 예정이다. 한파주의보가 내려졌던 연천·포천·가평·파주 등 4개 시·군은 한파경보로 변경되고,'),
 Document(metadata={'source': '/content/samlple.txt'}, page_content='한파주의보가 내려졌던 연천·포천·가평·파주 등 4개 시·군은 한파경보로 변경되고, 동두천·양주·의정부·남양주·여주·양평 등 6개 시·군은 한파경보가 발효된다.'),
 Document(metadata={'source': '/content/samlple.txt'}, page_content='또 광명, 과천, 안산, 시흥, 부천, 김포, 고양, 수원, 성남, 안양, 구리, 오산, 평택, 군포, 의왕, 하남, 용인, 이천, 안성, 화성, 광주 등 경기남부지역을 비롯한'),
 Document(metadata={'source': '/content/samlple.txt'}, page_content='평택, 군포, 의왕, 하남, 용인, 이천, 안성, 화성, 광주 등 경기남부지역을 비롯한 21개 지역에는 한파주의보를 발표됐다.'),
 Document(metadata={'source

#### 3. FAISS (Facebook AI Similarity Search, Vector Store) 적재하기

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings

retriever = FAISS.from_documents(
    texts, CohereEmbeddings(model="embed-multilingual-v3.0")
).as_retriever(search_kwargs={"k": 10})

#### 4. Retriever 생성 및 문서 검색하기

In [12]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [13]:
query = "한파에 대비해야할 것에는 어떤게 있나요?"
docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:

이·통장 등 협업을 통한 한파쉼터 운영상황 등 점검 ▲농작물 냉해 및 가축 동사 방지 등 사전 대비 ▲야외활동 자제, 부모님께 안부전화 하기 등 한파 행동요령과 안전수칙 적극
----------------------------------------------------------------------------------------------------
Document 2:

한파 피해 최소화를 위해 ▲노인, 노숙인 등 취약계층 안전확인 강화 ▲지역자율방재단, 이·통장 등 협업을 통한 한파쉼터 운영상황 등 점검 ▲농작물 냉해 및 가축 동사 방지 등
----------------------------------------------------------------------------------------------------
Document 3:

등 사전 대비 ▲야외활동 자제, 부모님께 안부전화 하기 등 한파 행동요령과 안전수칙 적극 홍보를 시군에 요청했다.
----------------------------------------------------------------------------------------------------
Document 4:

한편 한파특보가 내려질 경우 면역력이 약한 노약자와 어린이는 가급적 야외활동을 자제하는 등 급격한 기온 변화와 낮은 기온으로 인한 건강관리에 유의해야 한다. 또 난로와 전기장판
----------------------------------------------------------------------------------------------------
Document 5:

아울러 수도계량기·노출 수도관·보일러 등의 보온 상태를 점검해 동파에 대비하고, 온실과 축사에 난방장치를 가동해 농작물과 가축의 동사를 방지해야 한다.
---------------------------------------------------------------------------------------

## Cohere Rerank model - Reranker

#### ContextualCompressionRetriever 활용하여 Reranker 추가하기
- Base Retriever를 통해 검색된 문서를 한 번 더 필터링 및 압축하여 반환하는 Retriever
- 문맥 압축(Context Compression)이란, 문맥의 내용을 줄이거나 문맥을 완전히 삭제하는 것

In [14]:
# from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-multilingual-v3.0", top_n = 5) # rerank-english-v3.0
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "한파에 대비해야할 것에는 어떤게 있나요?"
)
pretty_print_docs(compressed_docs)

Document 1:

한편 한파특보가 내려질 경우 면역력이 약한 노약자와 어린이는 가급적 야외활동을 자제하는 등 급격한 기온 변화와 낮은 기온으로 인한 건강관리에 유의해야 한다. 또 난로와 전기장판
----------------------------------------------------------------------------------------------------
Document 2:

이·통장 등 협업을 통한 한파쉼터 운영상황 등 점검 ▲농작물 냉해 및 가축 동사 방지 등 사전 대비 ▲야외활동 자제, 부모님께 안부전화 하기 등 한파 행동요령과 안전수칙 적극
----------------------------------------------------------------------------------------------------
Document 3:

등 사전 대비 ▲야외활동 자제, 부모님께 안부전화 하기 등 한파 행동요령과 안전수칙 적극 홍보를 시군에 요청했다.
----------------------------------------------------------------------------------------------------
Document 4:

아울러 수도계량기·노출 수도관·보일러 등의 보온 상태를 점검해 동파에 대비하고, 온실과 축사에 난방장치를 가동해 농작물과 가축의 동사를 방지해야 한다.
----------------------------------------------------------------------------------------------------
Document 5:

한파 피해 최소화를 위해 ▲노인, 노숙인 등 취약계층 안전확인 강화 ▲지역자율방재단, 이·통장 등 협업을 통한 한파쉼터 운영상황 등 점검 ▲농작물 냉해 및 가축 동사 방지 등


## RAG

[Langchain Docs : create_stuff_documents_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html)

[Langchain Docs : create_retrieval_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval.create_retrieval_chain.html)

In [16]:
!pip install langchain-classic

In [17]:
# from langchain.chains import create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere

llm = ChatCohere()

system_prompt = (
    "Use the given context to answer the question in Korean. "
    "If you don't know the answer, say you don't know. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(compression_retriever, combine_docs_chain)

In [18]:
query = "한파에 대비해야할 것에는 어떤게 있나요?"
response = chain.invoke({"input": query})

In [19]:
response['input']

'한파에 대비해야할 것에는 어떤게 있나요?'

In [20]:
response['context']

[Document(metadata={'source': '/content/samlple.txt', 'relevance_score': 0.98439646}, page_content='한편 한파특보가 내려질 경우 면역력이 약한 노약자와 어린이는 가급적 야외활동을 자제하는 등 급격한 기온 변화와 낮은 기온으로 인한 건강관리에 유의해야 한다. 또 난로와 전기장판'),
 Document(metadata={'source': '/content/samlple.txt', 'relevance_score': 0.97879386}, page_content='이·통장 등 협업을 통한 한파쉼터 운영상황 등 점검 ▲농작물 냉해 및 가축 동사 방지 등 사전 대비 ▲야외활동 자제, 부모님께 안부전화 하기 등 한파 행동요령과 안전수칙 적극'),
 Document(metadata={'source': '/content/samlple.txt', 'relevance_score': 0.96391606}, page_content='등 사전 대비 ▲야외활동 자제, 부모님께 안부전화 하기 등 한파 행동요령과 안전수칙 적극 홍보를 시군에 요청했다.'),
 Document(metadata={'source': '/content/samlple.txt', 'relevance_score': 0.9567152}, page_content='아울러 수도계량기·노출 수도관·보일러 등의 보온 상태를 점검해 동파에 대비하고, 온실과 축사에 난방장치를 가동해 농작물과 가축의 동사를 방지해야 한다.'),
 Document(metadata={'source': '/content/samlple.txt', 'relevance_score': 0.7089184}, page_content='한파 피해 최소화를 위해 ▲노인, 노숙인 등 취약계층 안전확인 강화 ▲지역자율방재단, 이·통장 등 협업을 통한 한파쉼터 운영상황 등 점검 ▲농작물 냉해 및 가축 동사 방지 등')]

In [21]:
print(response['answer'])

한파에 대비해야 할 주요 사항은 다음과 같습니다:

1. **건강 관리**:  
   - 면역력이 약한 노약자와 어린이는 야외활동을 자제하고, 급격한 기온 변화와 낮은 기온에 대비해 건강 관리에 유의해야 합니다.  
   - 부모님께 안부 전화를 드리는 등 가족의 안전을 확인하는 것도 중요합니다.

2. **한파쉼터 운영 점검**:  
   - 지역자율방재단, 이·통장 등과의 협업을 통해 한파쉼터 운영 상황을 점검하고, 취약계층(노인, 노숙인 등)의 안전을 강화해야 합니다.

3. **농작물 및 가축 보호**:  
   - 농작물 냉해를 방지하고, 가축이 동사하지 않도록 온실과 축사에 난방장치를 가동해야 합니다.

4. **수도 시설 동파 방지**:  
   - 수도계량기, 노출 수도관, 보일러 등의 보온 상태를 점검해 동파를 예방해야 합니다.

5. **한파 행동요령 및 안전수칙 홍보**:  
   - 야외활동 자제, 난로와 전기장판 사용 시 주의사항 등 한파 행동요령과 안전수칙을 적극 홍보해야 합니다.

이러한 조치들을 통해 한파로 인한 피해를 최소화할 수 있습니다.


### 🔎 Reference

https://python.langchain.com/v0.2/docs/integrations/retrievers/cohere-reranker/